In [26]:
from models.DAX import DAX_quantile_regression, DAX_baseline
from models.energy import energy_quantile_regression
from functions import get_energy, get_DAX, merge_submissions, check_submission
from functions.prepare_data import split_time
from functions.evaluation import evaluate_horizon
from tqdm import tqdm
from functions import evaluate_dax
from functions import naive_ensemble
import pandas as pd
#import minmaxscaler
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, date, timedelta


In [27]:
energydata = get_energy.get()

  0%|          | 0/263 [00:00<?, ?it/s]/Users/stephantimpe/PycharmProjects/probabilistic-forecasting-challenge/functions/get_energy.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  energydata = pd.concat([energydata, pd.DataFrame(rawdata, columns=col_names)])
100%|██████████| 263/263 [00:27<00:00,  9.69it/s]


In [28]:
energydata.head()

,gesamt,weekday
date_time,,
2018-12-24 00:00:00,42.02925,0
2018-12-24 01:00:00,39.61025,0
2018-12-24 02:00:00,39.13875,0
2018-12-24 03:00:00,39.42100,0
2018-12-24 04:00:00,40.74775,0


Scale the data

In [29]:
# Creating a DataFrame
df = pd.DataFrame(energydata)

# Normalizing the 'gesamt' column
scaler = MinMaxScaler()
df['gesamt_normalized'] = scaler.fit_transform(df[['gesamt']])

df.head()  # Display the first few rows of the preprocessed DataFrame

,gesamt,weekday,gesamt_normalized
date_time,,,
2018-12-24 00:00:00,42.02925,0,0.504797
2018-12-24 01:00:00,39.61025,0,0.475743
2018-12-24 02:00:00,39.13875,0,0.470080
2018-12-24 03:00:00,39.42100,0,0.473470
2018-12-24 04:00:00,40.74775,0,0.489405


Split the data into train and test sets

In [30]:
# Assuming df has 'date_time' as its index and 'gesamt_normalized' as a column
# Extracting temporal features
df['hour'] = df.index.hour
df['day'] = df.index.day
df['month'] = df.index.month
df['week'] = df.index.weekday
# ... add other temporal features you find relevant

# Splitting the data into train and test sets chronologically
train_size = int(len(df) * 0.8)
train_df, test_df = df[:train_size], df[train_size:]

# Selecting the features and target for training
feature_columns = ['hour', 'day', 'month', 'weekday'] # add other columns if needed
train_X = train_df[feature_columns].values
train_y = train_df['gesamt'].values.reshape(-1, 1)
test_X = test_df[feature_columns].values
test_y = test_df['gesamt'].values.reshape(-1, 1)

# Shapes of the training and testing data
train_X.shape, train_y.shape, test_X.shape, test_y.shape



((35288, 4), (35288, 1), (8822, 4), (8822, 1))

# Train the LSTM model

In [31]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

# Assuming you have already prepared train_X, train_y, test_X, test_y as shown before

# Reshaping input data for LSTM [samples, time steps, features]
train_X_lstm = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))
test_X_lstm = np.reshape(test_X, (test_X.shape[0], 1, test_X.shape[1]))

Encoder

In [32]:
from keras.layers import Input, LSTM, RepeatVector
from keras.models import Model

# Define the input sequence
input_seq = Input(shape=(train_X_lstm.shape[1], train_X_lstm.shape[2]))

# Encoder LSTM
encoder_out, state_h, state_c = LSTM(50, return_state=True)(input_seq)
encoder_states = [state_h, state_c]


2024-01-04 22:33:23.803813: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-01-04 22:33:23.805318: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-01-04 22:33:23.806126: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Decoder

In [33]:
from keras.layers import Dense, TimeDistributed

# Set the number of time steps to predict
decoder_steps = train_X_lstm.shape[1]

# Decoder LSTM
decoder_lstm = LSTM(50, return_sequences=True)
decoder_out = decoder_lstm(RepeatVector(decoder_steps)(encoder_out), initial_state=encoder_states)

# Output layer
decoder_dense = TimeDistributed(Dense(5))  # Change 5 to the number of quantiles
decoder_outputs = decoder_dense(decoder_out)

# Define the model
model = Model(inputs=input_seq, outputs=decoder_outputs)


2024-01-04 22:33:23.940034: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-01-04 22:33:23.941081: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-01-04 22:33:23.942437: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Quantile Loss Function

In [34]:
def quantile_loss(quantiles, y_true, y_pred):
    e = y_true - y_pred
    losses = []
    for q in quantiles:
        loss = K.maximum(q * e, (q - 1) * e)
        losses.append(K.mean(loss, axis=-1))
    return K.mean(K.stack(losses, axis=-1), axis=-1)



Compile

In [35]:
quantiles = [0.025, 0.25, 0.5, 0.75, 0.975]
model.compile(optimizer='adam', loss=lambda y_true, y_pred: quantile_loss(quantiles, y_true, y_pred))


Train

In [36]:
model.fit(train_X_lstm, train_y, epochs=10, batch_size=32, verbose=1)


Epoch 1/10


2024-01-04 22:33:24.218632: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-01-04 22:33:24.220182: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-01-04 22:33:24.221170: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1103/1103 [==============================] - 5s 2ms/step - loss: 13.3007
Epoch 2/10
1103/1103 [==============================] - 2s 2ms/step - loss: 4.2052
Epoch 3/10
1103/1103 [==============================] - 2s 2ms/step - loss: 4.1733
Epoch 4/10
1103/1103 [==============================] - 2s 2ms/step - loss: 4.1752
Epoch 5/10
1103/1103 [==============================] - 3s 3ms/step - loss: 4.1750
Epoch 6/10
1103/1103 [==============================] - 2s 2ms/step - loss: 4.1757
Epoch 7/10
1103/1103 [==============================] - 2s 2ms/step - loss: 4.1747
Epoch 8/10
1103/1103 [==============================] - 2s 2ms/step - loss: 4.1741
Epoch 9/10
1103/1103 [==============================] - 2s 2ms/step - loss: 4.1741
Epoch 10/10
1103/1103 [==============================] - 2s 2ms/step - loss: 4.1736


In [37]:
predictions = model.predict(test_X_lstm)


2024-01-04 22:33:50.918088: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-01-04 22:33:50.919137: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-01-04 22:33:50.919928: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

276/276 [==============================] - 1s 622us/step


In [40]:
predictions

array([[[56.193165, 56.191467, 56.196503, 56.209404, 56.18016 ]],

       [[56.23614 , 56.23605 , 56.24045 , 56.254913, 56.223644]],

       [[56.268513, 56.2696  , 56.273552, 56.28921 , 56.256462]],

       ...,

       [[56.37169 , 56.3766  , 56.37927 , 56.398815, 56.36213 ]],

       [[56.37266 , 56.37757 , 56.38026 , 56.39986 , 56.363102]],

       [[56.373405, 56.37831 , 56.381023, 56.400654, 56.363846]]],
      dtype=float32)

In [41]:
from datetime import datetime, timedelta

def next_weekday(d, weekday):
    days_ahead = weekday - d.weekday()
    if days_ahead <= 0:  # Target day already happened this week
        days_ahead += 7
    return d + timedelta(days_ahead)

# Define your horizons in hours
horizons = [36, 40, 44, 60, 64, 68]

# Calculate the next Thursday at 00:00
now = datetime.now()
next_thursday = next_weekday(now, 3)  # 3 represents Thursday
next_thursday = next_thursday.replace(hour=0, minute=0, second=0, microsecond=0)

# Calculate future timestamps
future_timestamps = [next_thursday + timedelta(hours=h) for h in horizons]


In [42]:
# Function to extract features for a given timestamp
def extract_features_for_timestamp(timestamp, df):
    features = {
        'hour': timestamp.hour,
        'day': timestamp.day,
        'month': timestamp.month,
        'weekday': timestamp.weekday()
    }
    # If other features are needed, add them here

    # Convert to format similar to your training data
    feature_vector = [features[col] for col in feature_columns]
    return feature_vector

prediction_inputs = [extract_features_for_timestamp(ts, df) for ts in future_timestamps]


Reshape for LSTM

In [43]:
prediction_inputs = np.array(prediction_inputs).reshape(-1, 1, len(feature_columns))


In [44]:
predictions = model.predict(prediction_inputs)


2024-01-04 22:44:38.713526: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-01-04 22:44:38.714524: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-01-04 22:44:38.715302: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 1s 577ms/step


In [85]:
columns = [f'q{q}' for q in quantiles]
# Reshape predictions from 3D to 2D
predictions_reshaped = predictions.reshape(predictions.shape[0], -1)

# Create DataFrame with the reshaped predictions
df_predictions= pd.DataFrame()
df_predictions['forecast_date'] = next_thursday.strftime('%Y-%m-%d %H:%M:%S')* len(predictions_reshaped)
#df_predictions['forecast_date'] = [ts.strftime('%Y-%m-%d %H:%M:%S') for ts in future_timestamps]
df_predictions['target'] = 'energy'
df_predictions['horizon'] = [f'{h} hour' for h in horizons]
df_predictions[columns]=(predictions_reshaped)



In [90]:
# Static values DataFrame
df_static = pd.DataFrame({
    'forecast_date': [next_thursday.strftime('%Y-%m-%d')] * len(predictions_reshaped),
    'target': ['energy'] * len(predictions_reshaped),
    'horizon': [f'{h} hour' for h in horizons for _ in range(len(predictions_reshaped)//len(horizons))]
})

# DataFrame for predictions
df_predictions = pd.DataFrame(predictions_reshaped, columns=columns)

# Concatenate static DataFrame with predictions DataFrame
final_df = pd.concat([df_static, df_predictions], axis=1)


In [91]:
final_df

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-01-11,energy,36 hour,56.356960,56.362003,56.364002,56.381622,56.348495
1,2024-01-11,energy,40 hour,56.371048,56.376099,56.378548,56.397793,56.361744
2,2024-01-11,energy,44 hour,56.374722,56.379688,56.382347,56.401970,56.365242
3,2024-01-11,energy,60 hour,56.353516,56.358383,56.360477,56.377319,56.345455
4,2024-01-11,energy,64 hour,56.370300,56.375328,56.377796,56.396851,56.361088
5,2024-01-11,energy,68 hour,56.374577,56.379559,56.382202,56.401787,56.365124
